# Image Processing

In [2]:
%run ./notebook_init.py

import os
import cv2 as cv
import numpy as np
from shutil import rmtree

from glob import glob
from astropy.io import fits
from pathlib import Path
from core import DATA_FOLDER

In [2]:
file_ext = "fits"

image_folder_path = os.path.join(DATA_FOLDER, "connie_raw_dataset")
calibrated_img_path = os.path.join(image_folder_path, "calibrated_images")

In [3]:
calibrated_img_path_list = glob(os.path.join(calibrated_img_path, "*" + file_ext))

In [4]:
calibrated_img = []
mask = []
calibrated_img_filename = []

for img in calibrated_img_path_list:
    calibrated_img.append(fits.open(img)[0])
    calibrated_img_filename.append(Path(img).stem)

In [5]:
calibrated_img_list = []
for img in calibrated_img:
    calibrated_img_list.append(img.data)

In [6]:
calibrated_img_np = np.array(calibrated_img_list)
print(f"Shape calibrated_img_np = {np.shape(calibrated_img_np)}")

Shape calibrated_img_np = (1002, 1022, 420)


In [7]:
num_imgs = 10
nrows = 2
ncols = 5

In [8]:
new_mask = np.full(np.shape(calibrated_img_np[0]), False)

prescan = 8
overscan = 70
frame = 5

# new_mask[:, :prescan+frame] = True
# new_mask[:, -(overscan+frame):] = True
# new_mask[:frame, :] = True
# new_mask[-frame:, :] = True

new_mask[:, :frame] = True
new_mask[:, -frame:] = True
new_mask[:frame, :] = True
new_mask[-frame:, :] = True

In [9]:
def replace_negative(img):
    # 4 sigma (0.16) == 0.64
    return img.clip(0.64)

In [10]:
masked_border_img = []

for img in calibrated_img_np:
    masked_border_img.append(replace_negative(np.ma.masked_array(img, new_mask, fill_value=np.nan)))

### Global thresholding

In [11]:
factor = 3.745
threshold_ev = 10
threshold = threshold_ev/factor
print(f"Threshold in e- value equivalent to {threshold_ev}eV = {threshold}")

Threshold in e- value equivalent to 10eV = 2.67022696929239


### Saving cropped images

#### Opening (erosion + dilatation) and erosion

In [12]:
cropped_imgs_path = os.path.join(DATA_FOLDER, "connie_cropped_dataset", "unknown_label")

if os.path.exists(cropped_imgs_path):
    rmtree(cropped_imgs_path)

os.makedirs(cropped_imgs_path)

In [13]:
for idx, img in enumerate(masked_border_img):
    img_clip = masked_border_img[idx]
    img_original = masked_border_img[idx].copy()
    blurred_img = cv.GaussianBlur(img_clip, (3, 3), 0)
    _, img_thresh = cv.threshold(blurred_img, threshold, 255, cv.THRESH_BINARY)
    mask = cv.morphologyEx(img_thresh.astype(np.uint8), cv.MORPH_OPEN,
                           cv.getStructuringElement(cv.MORPH_CROSS,(3,3)))
    mask = cv.dilate(mask,
                     cv.getStructuringElement(cv.MORPH_CROSS,(1,1)), iterations=1)
    mask = cv.erode(mask,
                    cv.getStructuringElement(cv.MORPH_CROSS,(4, 4)), iterations=1)
    contours = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_TC89_KCOS)[0]
    contour_area_min = 2

    boxes = []
    for contour in contours:
        if (cv.contourArea(contour) >= contour_area_min):
            boxes.append(cv.boundingRect(contour))

    for box_idx, box in enumerate(boxes):
        top_left = (box[0], box[1])
        bottom_right = (box[0] + box[2], box[1] + box[3])
        cv.rectangle(img_original, top_left, bottom_right, (255, 0, 0), 5)
        cropped_img = img_clip[box[1]:box[1] + box[3], box[0]: box[0]+box[2]]
        padding_img = cv.copyMakeBorder(cropped_img, 30, 30, 30, 30,
                                        cv.BORDER_CONSTANT, None, value=0) 
        cv.imwrite(os.path.join(cropped_imgs_path, calibrated_img_filename[idx]
                                + "_" + str(box_idx) + ".png"), padding_img)